## 용언 추출

이 프로젝트의 목적은 주어진 전처리된 유튜브 콘텐츠 데이터에서 동사와 형용사와 같은 중요한 용언을 추출하는 것입니다. 용언은 문장에서 주로 동작이나 상태를 나타내므로, 이를 통해 콘텐츠의 주요 의미를 파악하고 분석할 수 있습니다.

### 사용된 라이브러리
- **konlpy**: 한국어 자연어 처리를 위한 라이브러리입니다.
- **mecab**: konlpy의 형태소 분석기 중 하나로, 빠르고 정확한 분석을 제공합니다.

### 구현 과정
1. **데이터 로드**: 전처리된 데이터셋을 로드합니다.
2. **형태소 분석**: `Mecab`을 이용하여 텍스트에서 용언을 추출합니다.
3. **용언 필터링**: 추출된 용언 중 동사 및 형용사만을 선택하여 분석에 활용합니다.

이 과정을 통해 유튜브 콘텐츠의 중요한 의미를 담고 있는 용언을 효과적으로 추출하여, 콘텐츠 분석 및 전략 수립에 기여할 수 있습니다.


In [1]:
from konlpy.tag import Mecab
import pandas as pd
from itertools import islice
import re

In [13]:
df = pd.read_csv("../data/video_sample.csv", encoding="utf-8-sig", index_col=0)
df.head()

,video_id,channel_id,video_title,video_description,video_tags,video_duration,video_published,video_category,video_info_card,video_with_ads,video_end_screen,video_cluster,crawled_date,year,month,day
0,JCqgRM-2_GU,UCU-wl-PZYxpWeDl1OIiE_hQ,어머니와 함께 방문 한 15 세 중학생 환자분의 포경수술 영상 (포경 소매법) C...,수술 적응증 상담 : 010-9190-7575\n카카오톡 상담 : gnjurolog...,"['강남비뇨기과', '박천진', '포경', '포경수술', '귀두포피', '위생관리']",75,2024-10-06,Education,0,0,0,-1,2024-10-07 17:37:39,2024,10,6
1,R0pRp-3S5Yo,UCm69xuysxVgNuRpmXCiXlPw,YURIAN (유리안) - FUCKING MY CITY 2024 ver. (Ft. ...,#YURIAN #유리안 #FUCKING_MY_CITY_2024 #Rredrain #...,[],177,2024-10-06,Music,0,0,0,-1,2024-10-07 17:37:57,2024,10,6
2,RsVKZrt2DSo,UCVcLszfUBN9rTmc0A0uDRSQ,대리모 사건이 소환한 합법화 논쟁 / KBC뉴스,불임이나 난임 부부가 브로커에게 돈을 주고 출산을 의뢰한 '대리모 범죄'가 14년 ...,"['kbc', 'kbc광주방송', '깨비씨', 'kbc 뉴스']",158,2024-10-06,News & Politics,0,0,0,-1,2024-10-07 17:38:23,2024,10,6
3,2CpgC1K7jNI,UCxCsxXsJPDMfjcXEEV09m9w,대포죽순이요????푸바오 핸펀몰카사건? 10.6 푸바오 이야기 FUBAO PANDA...,NaN,[],400,2024-10-06,Travel & Events,0,0,0,-1,2024-10-07 17:38:24,2024,10,6
4,C-jOF1sgdOg,UCYCb_mX7P1xKrB5iaT8y0Wg,"일본에 남겨진 한국의 문화유산【아시아의 숨은 혼, 백제를 가다 Part 2】",0:00 2012년 6월 18일 다카이다야마 고분\n1:43 2012년 6월 25일...,"['TJB', 'tjb', 'tjb대전방송', '엑스포로131', '엑스포로', '...",572,2024-10-07,Entertainment,0,0,0,-1,2024-10-07 17:38:24,2024,10,7


In [3]:
mecab = Mecab(dicpath='/home/suchoi/mecab/mecab-ko-dic-2.1.1-20180720')

In [4]:
# 맞는 품사 조합
match_poss = [["XR", "XSA+EP"], ["VV+EP", "EF"], ["VV+EP", "ETM"],
                ["NNG", "XSV+EF"], ["MAG", "XSA+ETM"], ["NNG", "XSA+EC"], 
                ["VV+EC", "VX+EP"], ["VV+EC", "VX+EF"], ["VV+ETM", "NNB", "VA"],
                ["NNG", "XSV+ETM"], ["NNG", "XSA+EP"], ["VA", "EP"],
                ["NNG", "XSV+EP"],  ["MAG", "VV+EC"], ["VV+ETM", "NNB", "VV"], 
                ["VV", "EP+EP"], ["NNG", "XSA+ETM"], ["VV+EC", "VX"],
                ["JKS", "VV+EC", "VX"], ["VV+EC", "VV+EC", "VX"], ["VV+EP", "EC"], 
                ['NNG', 'XSV+EC'], ['NNG', 'NNG', 'XSA+ETM']]

# 맞지 않는 품사 조합
not_match_poss = [["NNG", "XSA+ETM", "XPN"], ["NNG", "XSV+ETM", "NNG"], 
                    ["NNG", "XSV", "SY"], ["NNG", "VV+EC", "NNG"], ["JKS", "NNG", "XSV+ETM"],
                    ["NNG", "VCP+EP", "EF"], ["NNG", "XSV+ETM", "XPN"],
                    ['VCP+ETM', 'NNG', 'XSV+EC'], ['NNG', 'XSA+ETM', 'NNB']]

In [5]:
# 텍스트 정제 함수
def clean_text(text):
    # 해시태그 패턴
    hashtag_pattern = r'#\w+'
    
    # URL 패턴 (http, https, www로 시작하는 URL)
    url_pattern = r'https?://\S+|www\.\S+'
    
    # 해시태그와 URL 제거
    text_without_hashtags = re.sub(hashtag_pattern, ' ', text)
    clean_text = re.sub(url_pattern, ' ', text_without_hashtags)
    
    # 특수 문자 제거
    clean_text = clean_text.replace("|", " ").strip()
    clean_text = clean_text.replace("ㅣ", " ").strip()
    # 양쪽 공백 제거 및 [MASK]로 변환
    clean_text = clean_text.replace(" ", "[MASK]").strip()
    return clean_text  # 정제된 텍스트 반환


In [6]:
# 공통 패턴에 따라 필터링
def filter_by_common_pattern(data):
    result = list()  # 결과를 저장할 리스트
    for i, (word1, tag1) in enumerate(data):
        pass_tag = False  # 패스 태그 초기화
        for j, (word2, tag2) in enumerate(data):
            if i == j:  # 자기 자신은 제외
                continue
            if (set(tag1) and set(tag2)) and (len(tag1) < len(tag2)):  # 태그 비교
                pass_tag = True  # 패스 태그 설정
                break
        if not pass_tag:  # 패스 태그가 없으면 결과에 추가
            result.append((word1, tag1))
    return result  # 필터링된 결과 반환

In [7]:
def convert_to_da_form(text):
    # 입력값이 None인지 확인
    if text is None:
        return list()  # 빈 리스트나 기본 값을 반환
    
    text = clean_text(text)  # 텍스트를 정리하는 함수 호출

    # 품사 태깅
    pos = mecab.pos(text)  # Mecab을 사용하여 형태소 분석 및 품사 태깅 수행
    
    # 형태소 태그들만 추출
    tags = [tag for _, tag in pos]  # 각 형태소의 품사 태그만 추출

    # 필터링 결과를 저장할 리스트 초기화
    filtered_pos = []
    i = 0
    while i < len(pos):
        matched = False  # 매칭 여부 초기화
        # 일치하지 않는 품사 패턴과 비교
        for pattern in not_match_poss:
            pattern_length = len(pattern)  # 현재 패턴의 길이
            
            # 현재 태그와 패턴이 일치하는지 확인
            if tags[i:i + pattern_length] == pattern:
                i += pattern_length  # 패턴 길이만큼 건너뜀
                filtered_pos.append(("삭제됨", "DEL"))  # 삭제된 부분에 '삭제됨' 추가
                matched = True
                break
        if not matched:
            filtered_pos.append(pos[i])  # 일치하지 않는 품사는 그대로 추가
            i += 1

    verbs = []  # 변수를 저장할 리스트 초기화
    
    for match_pos in match_poss:
        # 슬라이딩 윈도우 방식으로 연속으로 존재하는 VV, EC, VX 찾기
        for i in range(len(filtered_pos) - len(match_pos) + 1):
            window = filtered_pos[i:i + len(match_pos)]  # 현재 슬라이딩 윈도우
            window_tags = [tag for _, tag in window]  # 현재 윈도우의 품사 태그
            
            # 현재 윈도우의 태그가 지정된 패턴과 일치하는지 확인
            if window_tags == match_pos:
                text = "".join([i[0] for i in window])  # 현재 윈도우의 형태소를 합침
                
                last_pos = window[-1][-1].split("+")[-1]  # 마지막 품사 태그
                if last_pos in ["EP", "VV", "VX", "VA", "XSV", "JX", "ETM"]:  # 특정 품사에 대해 처리
                    # 종결어미를 '다'로 변경
                    if text.endswith(tuple(["을", "는"])):
                        text = text[:-1]
                    text += "다"  # '다' 추가

                # 여러 조건에 따라 특정 어미를 변경
                if text.endswith(tuple(["합니다", "하겠다", "하였다", "했었다", "할까요", "해야다", 
                                        "한다면", "해서다", "해봤다", "합시다", "해라다", "하잖아", 
                                        "해도다", "한지다", "할까다", "합니까", "할런다", "합네다", 
                                        "할리다", "할려다", "할런지", "할려나", "할려고"])):
                    text = text[:-3] + "하다"
                elif text.endswith(tuple(["하고있다", "해버렸다", "해드려요", "한다면다", "할까요다"])):
                    text = text[:-4] + "하다"
                elif text.endswith(tuple(["하고있다", "해버렸다", "해드려요", "한다면다", "할까요다"])):
                    text = text[:-4] + "하다"
                elif text.endswith(tuple(["올라갈까"])):
                    text = text[:-4] + "오르다"
                elif text.endswith(tuple(["겨났는지다"])):
                    text = text[:-5] + "기다"
                elif text.endswith(tuple(["져버렸다"])):
                    text = text[:-4] + "졌다"
                elif text.endswith(tuple(["켰으니까"])):
                    text = text[:-4] + "키다"
                elif text.endswith(tuple(["한다", "한가", "했다", "해요", "해서", "할까", "해라", 
                                            "해다", "한데", "해줘", "할다", "항다", "햇다", "할지",
                                            "할려"])):
                    text = text[:-2] + "하다"
                elif text.endswith(tuple(["려다", "려줘"])):
                    text = text[:-2] + "리다"
                elif text.endswith(tuple(["와다"])):
                    text = text[:-2] + "왔다"
                elif text.endswith(tuple(["싶어"])):
                    text = text[:-2] + "싶다"
                elif text.endswith(tuple(["깼다"])):
                    text = text[:-2] + "깨다"
                elif text.endswith(tuple(["있어"])):
                    text = text[:-2] + "있다"
                elif text.endswith(tuple(["춰다", "췄다"])):
                    text = text[:-2] + "추다"
                elif text.endswith(tuple(["웠다", "워다"])):
                    text = text[:-2] + "우다"
                elif text.endswith(tuple(["녀다"])):
                    text = text[:-2] + "녔다"
                elif text.endswith(tuple(["갔다"])):
                    text = text[:-2] + "가다"
                elif text.endswith(tuple(["켜다"])):
                    text = text[:-2] + "켰다"
                elif text.endswith(tuple(["셨", "셧"])):
                    text = text[:-1] + "시다"
                elif text.endswith(tuple(["셨다"])):
                    text = text[:-2] + "시다"
                elif text.endswith(tuple(["셨습니까"])):
                    text = text[:-4] + "시다"
                elif text.endswith(tuple(["져다"])):
                    text = text[:-2] + "졌다"
                elif text.endswith(tuple(["탔다"])):
                    text = text[:-2] + "타다"
                elif text.endswith(tuple(["겨다"])):
                    text = text[:-2] + "기다"
                elif text.endswith(tuple(["겨났는다"])):
                    text = text[:-4] + "기다"
                elif text.endswith(tuple(["킨다", "킬까"])):
                    text = text[:-2] + "키다"
                elif text.endswith(tuple(["섰다"])):
                    text = text[:-2] + "서다"
                elif text.endswith(tuple(["줬다", "준다", "줄게"])):
                    text = text[:-2] + "주다"
                elif text.endswith(tuple(["냈다"])):
                    text = text[:-2] + "내다"
                elif text.endswith(tuple(["꼈다"])):
                    text = text[:-2] + "끼다"
                elif text.endswith(tuple(["꼈다다", "끼시다"])):
                    text = text[:-3] + "끼다"
                elif text.endswith(tuple(["음"])):
                    text = text[:-1] + "으다"
                elif text.endswith(tuple(["혀다", "혔다"])):
                    text = text[:-2] + "히다"
                elif text.endswith(tuple(["졌다", "진다"])):
                    text = text[:-2] + "지다"
                elif text.endswith(tuple(["려"])):
                    text = text[:-1] + "리다"
                elif text.endswith(tuple(["드다"])):
                    text = text[:-2] + "들다"
                elif text.endswith(tuple(["써다", "썼다"])):
                    text = text[:-2] + "쓰다"
                elif text.endswith(tuple(["러워", "런다"])):
                    text = text[:-2] + "럽다"
                elif text.endswith(tuple(["됨"])):
                    text = text[:-1] + "되다"
                elif text.endswith(tuple(["일다", "여다", "려다", "였다", "였어"])):
                    text = text[:-2] + "이다"
                elif text.endswith(tuple(["떠다"])):
                    text = text[:-2] + "뜨다"
                elif text.endswith(tuple(["띈다"])):
                    text = text[:-2] + "띄다"
                elif text.endswith(tuple(["둬라"])):
                    text = text[:-2] + "두다"
                elif text.endswith(tuple(["되고있다", "되어있다"])):
                    text = text[:-4] + "되다"
                elif text.endswith(tuple(["나버렸다"])):
                    text = text[:-4] + "났다"
                elif text.endswith(tuple(["뎌야한다"])):
                    text = text[:-4] + "디다"
                elif text.endswith(tuple(["려드리다", "릴게요다"])):
                    text = text[:-4] + "리다"
                elif text.endswith(tuple(["들으며다"])):
                    text = text[:-4] + "듣다"
                elif text.endswith(tuple(["됩니다", "된다면", "될지다", "된다던"])):
                    text = text[:-3] + "되다"
                elif text.endswith(tuple(["이었다", "여줄까", "여봐요"])):
                    text = text[:-3] + "이다"
                elif text.endswith(tuple(["폈는데다", "펴봤더니"])):
                    text = text[:-4] + "피다"
                elif text.endswith(tuple(["려드릴께요", "려드릴게요"])):
                    text = text[:-5] + "리다"
                elif text.endswith(tuple(["여드릴께요", "여드릴게요"])):
                    text = text[:-5] + "이다"
                elif text.endswith(tuple(["라갈까"])):
                    text = text[:-3] + "르다"
                elif text.endswith(tuple(["봅니다", "볼게요", "볼까요"])):
                    text = text[:-3] + "보다"
                elif text.endswith(tuple(["타봤다"])):
                    text = text[:-3] + "타다"
                elif text.endswith(tuple(["로운다"])):
                    text = text[:-3] + "롭다"
                elif text.endswith(tuple(["진다면"])):
                    text = text[:-3] + "지다"
                elif text.endswith(tuple(["개놓다"])):
                    text = text[:-3] + "개다"
                elif text.endswith(tuple(["사볼까"])):
                    text = text[:-3] + "사다"
                elif text.endswith(tuple(["다운다"])):
                    text = text[:-3] + "답다"
                elif text.endswith(tuple(["췄다"])):
                    text = text[:-2] + "추다"
                elif text.endswith(tuple(["춰줄게", "춰줄께"])):
                    text = text[:-3] + "추다"
                elif text.endswith(tuple(["립니다", "릴까다", "려서다", "려볼까"])):
                    text = text[:-3] + "리다"
                elif text.endswith(tuple(["겨봤다", "겼으니"])):
                    text = text[:-3] + "기다"
                elif text.endswith(tuple(["져지다"])):
                    text = text[:-3] + "졌다"
                elif text.endswith(tuple(["워봤다", "웠더니"])):
                    text = text[:-3] + "우다"
                elif text.endswith(tuple(["나봤다"])):
                    text = text[:-3] + "났다"
                elif text.endswith(tuple(["뎌야다"])):
                    text = text[:-3] + "디다"
                elif text.endswith(tuple(["웁시다"])):
                    text = text[:-3] + "울다"
                elif text.endswith(tuple(["않아요"])):
                    text = text[:-3] + "않다"
                elif text.endswith(tuple(["켜야다"])):
                    text = text[:-3] + "키다"
                elif text.endswith(tuple(["났다"])):
                    text = text[:-2] + "나다"
                elif text.endswith(tuple(["켰다", "킨다", "킬다"])):
                    text = text[:-2] + "키다"
                elif text.endswith(tuple(["샀다"])):
                    text = text[:-2] + "사다"
                elif text.endswith(tuple(["켜"])):
                    text = text[:-1] + "키다"
                elif text.endswith(tuple(["해"])):
                    text = text[:-1] + "하다"
                elif text.endswith(tuple(["봐"])):
                    text = text[:-1] + "보다"
                elif text.endswith(tuple(["기", "냐", "네", "죠", "어", "지", "나", "고", "니"])):
                    text = text[:-1] + "다"
                elif text.endswith(tuple(["으니까다", "잖아요다"])):
                    text = text[:-4] + "다"
                elif text.endswith(tuple(["러운다", "러우다"])):
                    text = text[:-3] + "럽다"
                elif text.endswith(tuple(["네요", "셨다", "슴다", "어요", "다구", "라다", "렸나", "을까", \
                    "다면", "는데", "잖아", "다고", "었다", "어다", "쥬다", "나다", "고다", "았다", "겠다", \
                    "은다" "다다", "던다", "나요", "줘요", "다가", "지만", "더니", "였니", "는다", "습니", \
                    "으면", "습다", "시다", "으다"])):
                    text = text[:-2] + "다"
                elif text.endswith(tuple(["습니다", "다고다", "읍니다", "더니다", "는데요", \
                    "냐구요", "아겠네", "지만다", "는데다", "구나다", "다가다", "리면다", "을까다", \
                    "냐고요", "으시다"])):
                    text = text[:-3] + "다"
                elif text.endswith(tuple(["됐다", "된다", "될다", "될까", "돼다"])):
                    text = text[:-2] + "되다"
                elif text.endswith(tuple(["돼"])):
                    text = text[:-2] + "되다"
                    
                ### 2번째 처리(과거시제)
                # 다양한 과거형 어미에 대한 처리
                if text.endswith(tuple(["합니다", "한데다", "할게요"])):
                    text = text[:-3] + "하다"
                elif text.endswith(tuple(["줬다"])):
                    text = text[:-2] + "주다"
                elif text.endswith(tuple(["다다", "는다", "더다"])):
                    text = text[:-2] + "다"
                elif text.endswith(tuple(["려드리다", "려야하다", "렸잖아요"])):
                    text = text[:-4] + "리다"
                elif text.endswith(tuple(["춰다", "췄다"])):
                    text = text[:-2] + "추다"
                elif text.endswith(tuple(["립니다", "렸더니", "릴게요"])):
                    text = text[:-3] + "리다"
                elif text.startswith(tuple(["지으"])):
                    text = "짓" + text[2:]
                elif text.endswith(tuple(["섰다"])):
                    text = text[:-2] + "서다"
                elif text.startswith(tuple(["아무리"])):
                    text = text[3:]
                elif text.endswith(tuple(["렸다", "랐다", "린다"])):
                    text = text[:-2] + "리다"
                elif text.endswith(tuple(["였다"])):
                    text = text[:-2] + "이다"
                elif text.endswith(tuple(["웠다"])):
                    text = text[:-2] + "우다"
                elif text.endswith(tuple(["떴다"])):
                    text = text[:-2] + "뜨다"
                elif text.endswith(tuple(["갔다"])):
                    text = text[:-2] + "가다"
                elif text.endswith(tuple(["겼다"])):
                    text = text[:-2] + "기다"
                elif text.endswith(tuple(["켰다", "켜야"])):
                    text = text[:-2] + "키다"
                elif text.endswith(tuple(["쳤다", "친다"])):
                    text = text[:-2] + "치다"
                elif text.endswith(tuple(["혀다", "혔다"])):
                    text = text[:-2] + "히다"
                elif text.endswith(tuple(["쳤지만"])):
                    text = text[:-3] + "치다"
                elif text.endswith(tuple(["끼시다"])):
                    text = text[:-3] + "끼다"
                elif text.endswith(tuple(["났다"])):
                    text = text[:-2] + "나다"
                elif text.endswith(tuple(["탔다"])):
                    text = text[:-2] + "타다"
                elif text.endswith(tuple(["봤다", "봐다"])):
                    text = text[:-2] + "보다"
                elif text.endswith(tuple(["졌다", "졌나"])):
                    text = text[:-2] + "지다"
                elif text.endswith(tuple(["냈다"])):
                    text = text[:-2] + "내다"
                elif text.endswith(tuple(["왔다", "왔나", "와라"])):
                    text = text[:-2] + "오다"
                elif text.endswith(tuple(["꿨다"])):
                    text = text[:-2] + "꾸다"
                elif text.endswith(tuple(["뤘다"])):
                    text = text[:-2] + "루다"
                elif text.endswith(tuple(["해"])):
                    text = text[:-1] + "하다"
                elif text.endswith(tuple(["갔었다", "갔다다", "갑니다"])):
                    text = text[:-3] + "가다"
                elif text.endswith(tuple(["났었다", "났구나", "났지만"])):
                    text = text[:-3] + "나다"
                elif text.endswith(tuple(["오시다"])):
                    text = text[:-3] + "오다"
                elif text.endswith(tuple(["올랐다"])):
                    text = text[:-3] + "오르다"
                elif text.endswith(tuple(["몰랐다"])):
                    text = text[:-3] + "모르다"
                elif text.endswith(tuple(["늘렀다"])):
                    text = text[:-3] + "누르다"
                elif text.endswith(tuple(["질렀다"])):
                    text = text[:-3] + "지르다"
                elif text.endswith(tuple(["질렀길래"])):
                    text = text[:-4] + "지르다"
                elif text.endswith(tuple(["질러야지다"])):
                    text = text[:-5] + "지르다"
                elif text.endswith(tuple(["불렀다"])):
                    text = text[:-3] + "부르다"
                elif text.endswith(tuple(["샀다"])):
                    text = text[:-2] + "사다"
                elif text.endswith(tuple(["찼다"])):
                    text = text[:-2] + "차다"
                elif text.endswith(tuple(["뗐다"])):
                    text = text[:-2] + "떼다"
                elif text.endswith(tuple(["됐다", "된다", "될다", "될까", "돼다", "돼야"])):
                    text = text[:-2] + "되다"
                elif text.endswith(tuple(["한다", "했다", "해도", "해야", "할다"])):
                    text = text[:-2] + "하다"

                # '하다', '보다', '받다' 등 특정 불변 동사는 제외
                if text not in ["하다", "보다", "받다", "되다", "봤다", "나다", "났다", 
                    "가다", "있다", "없다", "많다", "그렇다", "그랬다", "저랬다"]:
                    if text.strip().endswith("다"):
                        verbs.append((text, match_pos))  # 변수를 리스트에 추가
    verbs = filter_by_common_pattern(verbs)  # 공통 패턴으로 필터링

    return verbs

In [17]:
for video_title in df.video_title:
    verb = convert_to_da_form(video_title)
    if len(verb) > 0:
        print(verb)

[('소환하다', ['NNG', 'XSV+ETM'])]
[('공분하다', ['NNG', 'XSA+ETM'])]
[('밝혀내다', ['VV+EC', 'VX'])]
[('발표하다', ['NNG', 'XSV+ETM'])]
[('고소당하다', ['NNG', 'XSV+ETM'])]
[('소개하다', ['NNG', 'XSV+EF'])]
[('영업하다', ['NNG', 'XSV+EC'])]
[('함께하다', ['MAG', 'XSA+ETM'])]
[('달성하다', ['NNG', 'XSV+ETM']), ('발표하다', ['NNG', 'XSV+ETM'])]
